# Regression tests

In [1]:
import tohu
from tohu import *
from utils import print_generated_sequence

In [2]:
# NBVAL_IGNORE_OUTPUT
tohu.__version__

'v0.6.0'

## Resetting one generator should not reset others of the same type

Let's create two generators of the same type.

In [3]:
class FoobarGenerator(CustomGenerator):
    a = Integer(low=1000, high=3000)
    b = Sequential(prefix="Foo_", digits=2)
    c = Float(low=1.0, high=4.0)

In [4]:
g1 = FoobarGenerator()
g2 = FoobarGenerator()

The random generators which produce the attributes of items generated by `g1` and `g2` must be independent of each other. This regression test checks this by making sure that `g1.reset()` does not interfere with `g2.reset()`. In particular, the two outputs below should be identical (because the final call `g1.reset(12345)` should not interfere with the previous call `g2.reset(9999)`).

In [5]:
g1.reset(seed=12345)
g2.reset(seed=9999)
print_generated_sequence(g1, num=3, sep='\n')
print_generated_sequence(g2, num=3, sep='\n')

print("-------------------------------------------------")

g1.reset(seed=12345)
g2.reset(seed=9999)
g1.reset(seed=12345)
print_generated_sequence(g1, num=3, sep='\n')
print_generated_sequence(g2, num=3, sep='\n')

Generated sequence:
Foobar(a=2412, b='Foo_01', c=1.733764868146123)
Foobar(a=2542, b='Foo_02', c=2.6100055602718233)
Foobar(a=2888, b='Foo_03', c=3.658587983373506)
Generated sequence:
Foobar(a=1403, b='Foo_01', c=3.0691459038129345)
Foobar(a=2827, b='Foo_02', c=3.6241864318459376)
Foobar(a=1032, b='Foo_03', c=3.898644498622002)
-------------------------------------------------
Generated sequence:
Foobar(a=2412, b='Foo_01', c=1.733764868146123)
Foobar(a=2542, b='Foo_02', c=2.6100055602718233)
Foobar(a=2888, b='Foo_03', c=3.658587983373506)
Generated sequence:
Foobar(a=1403, b='Foo_01', c=3.0691459038129345)
Foobar(a=2827, b='Foo_02', c=3.6241864318459376)
Foobar(a=1032, b='Foo_03', c=3.898644498622002)


## Sub-generators of custom generator should be initialised with separate seeds

Each custom generator can be re-initialised using a given `seed`. Internally, this re-initialises each of the constituent generators. This test checks that different seeds are used for each of the individual generators, even if they are of the same type (otherwise the values produced for each attribute would be the same). In particular, the sequences produced for `a1`, `a2`, `a3` below should be different (and similarly for (`b1`, `b2`, `b3`).

In [6]:
class FoobarGenerator(CustomGenerator):
    a1 = HashDigest(length=8)
    a2 = HashDigest(length=8)
    a3 = HashDigest(length=8)
    
    b1 = Integer(low=0, high=1000)
    b2 = Integer(low=0, high=1000)
    b3 = Integer(low=0, high=1000)

In [7]:
g = FoobarGenerator()

In [8]:
g.reset(seed=99999)
print_generated_sequence(g, num=10, sep='\n')

Generated sequence:
Foobar(a1='449D838A', a2='4A634C13', a3='8A1B2368', b1=834, b2=830, b3=132)
Foobar(a1='95B11A02', a2='FC841110', a3='045643D7', b1=224, b2=498, b3=113)
Foobar(a1='E10FC5D6', a2='5EA42C2D', a3='E8A57CF4', b1=461, b2=194, b3=931)
Foobar(a1='B7E7446B', a2='CF779867', a3='A48AD3A0', b1=98, b2=439, b3=823)
Foobar(a1='014500FF', a2='D60DC72D', a3='6F3C525F', b1=792, b2=475, b3=33)
Foobar(a1='1852E7F2', a2='552ED478', a3='958B86F4', b1=488, b2=612, b3=539)
Foobar(a1='5B20DE66', a2='6C1703F6', a3='082D324E', b1=517, b2=366, b3=595)
Foobar(a1='7533D0C9', a2='FC9D3682', a3='33EA75F3', b1=43, b2=444, b3=540)
Foobar(a1='60002EE4', a2='A05CE1D6', a3='90312DFF', b1=236, b2=605, b3=109)
Foobar(a1='9AC16FE8', a2='0C309EF7', a3='A9A85C27', b1=608, b2=339, b3=139)


## NumpyRandomGenerator

This is a quick sanity check that resetting a `NumpyRandomGenerator` really fully resets the state and leads to independent runs.

In [9]:
g = NumpyRandomGenerator(method="normal", loc=3.0, scale=5.0)
g.reset(seed=12345); print_generated_sequence(g, num=4)
g.reset(seed=99999); print_generated_sequence(g, num=4)

Generated sequence: 1.9764617025764353, 5.394716690287741, 0.40280642471630923, 0.22134847826254989
Generated sequence: 6.120470228691696, 9.374815429760194, -5.298018043341111, 5.539749427572133


In [10]:
g1 = NumpyRandomGenerator(method="normal", loc=3.0, scale=5.0)
g2 = NumpyRandomGenerator(method="normal", loc=3.0, scale=5.0)
g1.reset(seed=12345)
g2.reset(seed=99999)
print_generated_sequence(g1, num=4)
print_generated_sequence(g2, num=4)

Generated sequence: 1.9764617025764353, 5.394716690287741, 0.40280642471630923, 0.22134847826254989
Generated sequence: 6.120470228691696, 9.374815429760194, -5.298018043341111, 5.539749427572133
